In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt

In [3]:
mnist = input_data.read_data_sets('MNIST_data/',one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [4]:
#have to define some functions for different layers

In [5]:
# input -> conv->max pooling->conv -> max pooling -->conv flat-> fully connected -> relu-> dropout-> fully_connected->prediction

In [6]:
train_x = mnist.train.images
train_y = mnist.train.labels

In [7]:
x = tf.placeholder(tf.float32,[None,784])
y_true = tf.placeholder(tf.float32,[None,10])

In [8]:
input_x = tf.reshape(x,[-1,28,28,1])

In [9]:
#now creating values
w_1 = tf.Variable(tf.truncated_normal(shape=[6,6,1,32],stddev=0.1))
b_1 = tf.Variable(tf.constant(0.1,shape=[32]))
conv_1 = tf.nn.relu(tf.nn.conv2d(input_x,w_1,strides=[1,1,1,1],padding='SAME')+b_1)
conv_1_pool = tf.nn.max_pool(conv_1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [10]:
w_2 = tf.Variable(tf.truncated_normal(shape=[6,6,32,64],stddev=0.1))
b_2 = tf.Variable(tf.constant(0.1,shape=[64]))
conv_2 = tf.nn.relu(tf.nn.conv2d(conv_1_pool,w_2,strides=[1,1,1,1],padding='SAME')+b_2)
conv_2_pool = tf.nn.max_pool(conv_2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [11]:
conv_flat = tf.reshape(conv_2_pool,[-1,7*7*64])

In [12]:
w_3 = tf.Variable(tf.truncated_normal(shape=[int(conv_flat.get_shape()[1]),1024],stddev=0.1) )
b_3 = tf.Variable(tf.constant(0.1,shape=[1024]))
flat_one_layer = tf.nn.relu(tf.matmul(conv_flat,w_3) + b_3)

In [13]:
hold_prob = tf.placeholder(tf.float32)

In [14]:
dropout_layer = tf.nn.dropout(flat_one_layer,keep_prob=hold_prob)

In [15]:
w_4 = tf.Variable(tf.truncated_normal(shape=[1024,10],stddev=0.1))
b_4 = tf.Variable(tf.constant(0.1,shape=[10]))
y_pred = tf.matmul(dropout_layer,w_4) + b_4

In [16]:
#defining loss function
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_pred,labels=y_true)

In [17]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

In [18]:
init  = tf.global_variables_initializer()

In [19]:
steps = 1000

In [20]:
with tf.Session() as sess:
    sess.run(init)
    for i in range(steps):
        batch_x,batch_y =mnist.train.next_batch(50)
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.5})  
        if i%100==0:
            print("step: {}".format(i))
            print("Accuracy: ")
            matches = tf.equal(tf.argmax(y_true,1),tf.argmax(y_pred,1))
            acc = tf.reduce_mean(tf.cast(matches,tf.float32))
            
            print(sess.run(acc,feed_dict={x:mnist.test.images[:1000],y_true:mnist.test.labels[:1000],hold_prob:1.0}))

step: 0
Accuracy: 
0.149
step: 100
Accuracy: 
0.919
step: 200
Accuracy: 
0.958
step: 300
Accuracy: 
0.964
step: 400
Accuracy: 
0.973
step: 500
Accuracy: 
0.97
step: 600
Accuracy: 
0.968
step: 700
Accuracy: 
0.97
step: 800
Accuracy: 
0.983
step: 900
Accuracy: 
0.979


In [21]:
#now using functions`

In [23]:
#mnist is a dataset for mnist

In [24]:
def init_weights(shape):
    init_weight = tf.truncated_normal(shape=shape,stddev=0.1)
    return tf.Variable(init_weight)

In [38]:
def init_bias(shape):
    init_biases = tf.constant(0.1,shape=shape)
    return tf.Variable(init_biases)

In [39]:
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding="SAME")

In [40]:
def max_pool_2by2(input_x):
    return tf.nn.max_pool(input_x,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")

In [48]:
def convolutional_layer(input_x,shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x,W) + b)

In [60]:
def normal_layer(input_x,size):
    W = init_weights([int(input_x.get_shape()[1]),size])
    b = init_bias([size])
    return tf.matmul(input_x,W) + b

In [61]:
#now defining the layer
#first layer
convo_1 = convolutional_layer(input_x,[6,6,1,32])
convo_1_pool = max_pool_2by2(convo_1)

In [62]:
#second layer
convo_2 = convolutional_layer(convo_1_pool,[6,6,32,64])
convo_2_pool = max_pool_2by2(convo_2)

In [63]:
convo_flat = tf.reshape(convo_2_pool,[-1,7*7*64])
flat_one_layer = tf.nn.relu(normal_layer(convo_flat,1024))

In [65]:
dropout_layer = tf.nn.dropout(flat_one_layer,keep_prob=hold_prob)
y_pred_1 = normal_layer(dropout_layer,10)

In [66]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_pred,labels=y_true)

In [67]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

In [69]:
steps = 1000
init = tf.global_variables_initializer()

In [73]:
with tf.Session() as sess:
    sess.run(init)
    for i in range(steps):
        batch_x,batch_y = mnist.train.next_batch(50)
        sess.run(train,feed_dict={x:batch_x,y_true:batch_y,hold_prob:0.6})
        if i%100==0: 
            print('epoch: {}'.format(i))
            print('Accuracy: ')
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
            acc = tf.reduce_mean(tf.cast(matches,tf.float32))
            print(sess.run(acc,feed_dict={x:mnist.test.images[:1000],y_true:mnist.test.labels[:1000],hold_prob:1.0}))

epoch: 0
Accuracy: 
0.219
epoch: 100
Accuracy: 
0.938
epoch: 200
Accuracy: 
0.962
epoch: 300
Accuracy: 
0.962
epoch: 400
Accuracy: 
0.97
epoch: 500
Accuracy: 
0.976
epoch: 600
Accuracy: 
0.974
epoch: 700
Accuracy: 
0.976
epoch: 800
Accuracy: 
0.979
epoch: 900
Accuracy: 
0.985
